In [8]:
import itertools
import numpy as np
# PyTorch
import torch
import torchvision

In [2]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2
# Importing our custom module(s)
import utils
from posteriors.swag import SWAG

In [6]:
checkpoint_path = '/cluster/home/eharve06/understanding-SWAG/experiments/swag_ImageNet_v2_torchvision/swag_epochs=30_K=20_lr=0.01_no_cov_factor=False_random_state=1001.pt'
clf_weight_decays = [0.01, 0.001, 0.0001, 1e-05, 1e-06, 0.0]
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/home/eharve06/understanding-SWAG/experiments/retrained_FS_CIFAR-10'
lr_0s = [0.1, 0.01, 0.001, 0.0001]
ns = [1000]
random_states = [1001, 2001, 3001]
wandb_project = 'retrained_FS_CIFAR-10'
count = -1
for clf_weight_decay, lr_0, n, random_state in itertools.product(clf_weight_decays, lr_0s, ns, random_states):
    model_name = f'FSEnsemblePrior_clf_weight_decay={clf_weight_decay}_lambda=0.1_lr_0={lr_0}_n={n}_random_state={random_state}'
    count += 1
    print(f'    "python ../src/CIFAR-10_main-Copy1.py --checkpoint_path=\'{checkpoint_path}\' --clf_weight_decay={clf_weight_decay} --dataset_path=\'{dataset_path}\' --experiments_path=\'{experiments_path}\' --lr_0={lr_0} --model_name=\'{model_name}\' --n={n} --random_state={random_state} --sigma=0.1 --tune --wandb --wandb_project=\'{wandb_project}\'"')
print(count)

    "python ../src/CIFAR-10_main-Copy1.py --checkpoint_path='/cluster/home/eharve06/understanding-SWAG/experiments/swag_ImageNet_v2_torchvision/swag_epochs=30_K=20_lr=0.01_no_cov_factor=False_random_state=1001.pt' --clf_weight_decay=0.01 --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/home/eharve06/understanding-SWAG/experiments/retrained_FS_CIFAR-10' --lr_0=0.1 --model_name='FSEnsemblePrior_clf_weight_decay=0.01_lambda=0.1_lr_0=0.1_n=1000_random_state=1001' --n=1000 --random_state=1001 --sigma=0.1 --tune --wandb --wandb_project='retrained_FS_CIFAR-10'"
    "python ../src/CIFAR-10_main-Copy1.py --checkpoint_path='/cluster/home/eharve06/understanding-SWAG/experiments/swag_ImageNet_v2_torchvision/swag_epochs=30_K=20_lr=0.01_no_cov_factor=False_random_state=1001.pt' --clf_weight_decay=0.01 --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/home/eharve06/understanding-SWAG/experiments/retrained_FS_CIFAR-10' --lr

python ../src/CIFAR-10_main-Copy1.py --checkpoint_path='/cluster/home/eharve06/understanding-SWAG/experiments/swag_ImageNet_v2_torchvision/swag_epochs=30_K=20_lr=0.01_no_cov_factor=False_random_state=1001.pt' --clf_weight_decay=0.0 --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/home/eharve06/understanding-SWAG/experiments/retrained_FS_CIFAR-10' --lr_0=0.1 --model_name='FSEnsemblePrior_clf_weight_decay=0.0_lambda=1.0_lr_0=0.1_n=1000_random_state=1001' --n=1000 --random_state=1001 --sigma=1.0 --tune --wandb --wandb_project='function-test'

python ../src/CIFAR-10_main-Copy1.py --checkpoint_path='/cluster/home/eharve06/understanding-SWAG/experiments/swag_ImageNet_v2_torchvision/swag_epochs=30_K=20_lr=0.01_no_cov_factor=False_random_state=1001.pt' --clf_weight_decay=0.0 --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/home/eharve06/understanding-SWAG/experiments/retrained_FS_CIFAR-10' --lr_0=0.1 --model_name='FSEnsemblePrior_clf_weight_decay=0.0_lambda=10.0_lr_0=0.1_n=1000_random_state=1001' --n=1000 --random_state=1001 --sigma=10.0 --tune --wandb --wandb_project='function-test'

In [11]:
k = 5
cov_diag = torch.load('/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior/resnet50_ssl_prior_variance.pt', map_location=torch.device('cpu'))
cov_factor = torch.load('/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior/resnet50_ssl_prior_covmat.pt', map_location=torch.device('cpu'))
loc = torch.load('/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior/resnet50_ssl_prior_mean.pt', map_location=torch.device('cpu'))
# $\Sigma = \frac{1}{2} ( \Sigma_{\text{diag}} + \Sigma_{\text{LR}} )$
cov_diag = (1/2)*cov_diag
cov_factor = np.sqrt(1/2)*cov_factor
# $\Sigma_{\text{LR}} = \frac{1}{k-1} Q Q^T$
cov_factor = np.sqrt(1/(k-1))*cov_factor
mvn = torch.distributions.lowrank_multivariate_normal.LowRankMultivariateNormal(loc, cov_factor.t(), cov_diag)
mvn.sample()

tensor([ 1.1217e-02,  4.2486e-03,  3.0997e-04,  ..., -2.7197e+00,
        -4.8237e+00, -4.1855e+00])